# Hardware Emulation and PYNQ

This notebook shows how to set up PYNQ to use the Vitis hardware emulator rather than real hardware. The starting point is to build the emulation xclbin file which follows the exact same steps as for building one for real hardware. You can see the *Building with Vitis* notebook alongside this one for more details on the commands used.

The only change we need to make is to pass `-t hw_emu` to the `v++` tool.

In [1]:
%%writefile oop.cpp

#include <ap_int.h>

class Node {
public:
    ap_int<32> id;
    ap_int<32> inV;
    ap_int<32> outV;
    ap_int<32> value;

    ap_int<32> operate(Node* nodes) {
        return nodes[outV].value;
    }
};

extern "C" {
    void oop(Node* nodes, ap_int<32> start_id, ap_int<32>* out) {
        #pragma HLS INTERFACE m_axi port=nodes offset=slave bundle=gmem
        #pragma HLS INTERFACE s_axilite port=start_id
        #pragma HLS INTERFACE m_axi port=out offset=slave bundle=gmem
        #pragma HLS INTERFACE s_axilite port=out
        #pragma HLS INTERFACE s_axilite port=return

        ap_int<32> current_id = start_id;

        *out = nodes[current_id].operate(nodes);
    }
}




Overwriting oop.cpp


In [2]:
import glob

platform = glob.glob("/opt/xilinx/platforms/*/*.xpfm")[0]

In [3]:
!v++ -c oop.cpp -t hw_emu --kernel oop -f $platform -o oop.xo

Option Map File Used: '/tools/Xilinx/Vitis/2022.1/data/vitis/vpp/optMap.xml'

****** v++ v2022.1 (64-bit)
  **** SW Build 3524075 on 2022-04-13-17:42:45
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.

INFO: [v++ 60-1306] Additional information associated with this v++ compile can be found at:
	Reports: /home/jun/pynq-notebooks/4-building-and-emulation/_x/reports/oop
	Log files: /home/jun/pynq-notebooks/4-building-and-emulation/_x/logs/oop
Running Dispatch Server on port: 37289
INFO: [v++ 60-1548] Creating build summary session with primary output /home/jun/pynq-notebooks/4-building-and-emulation/oop.xo.compile_summary, at Thu Jul 20 04:56:11 2023
INFO: [v++ 60-1316] Initiating connection to rulecheck server, at Thu Jul 20 04:56:11 2023
INFO: [v++ 60-1315] Creating rulecheck session with output '/home/jun/pynq-notebooks/4-building-and-emulation/_x/reports/oop/v++_compile_oop_guidance.html', at Thu Jul 20 04:56:13 2023
INFO: [v++ 60-895]   Target platform: /opt/xilinx/platf

The output is the `fir.xo` file which contains the compiled kernel

In [4]:
!v++ -l -t hw_emu -o oop_emu.xclbin -f $platform oop.xo

Option Map File Used: '/tools/Xilinx/Vitis/2022.1/data/vitis/vpp/optMap.xml'

****** v++ v2022.1 (64-bit)
  **** SW Build 3524075 on 2022-04-13-17:42:45
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.

INFO: [v++ 60-1306] Additional information associated with this v++ link can be found at:
	Reports: /home/jun/pynq-notebooks/4-building-and-emulation/_x/reports/link
	Log files: /home/jun/pynq-notebooks/4-building-and-emulation/_x/logs/link
Running Dispatch Server on port: 45693
INFO: [v++ 60-1548] Creating build summary session with primary output /home/jun/pynq-notebooks/4-building-and-emulation/oop_emu.xclbin.link_summary, at Thu Jul 20 04:56:57 2023
INFO: [v++ 60-1316] Initiating connection to rulecheck server, at Thu Jul 20 04:56:57 2023
INFO: [v++ 60-1315] Creating rulecheck session with output '/home/jun/pynq-notebooks/4-building-and-emulation/_x/reports/link/v++_link_oop_emu_guidance.html', at Thu Jul 20 04:56:59 2023
INFO: [v++ 60-895]   Target platform: /opt/xilinx

## Setting up the emulator

Now we have a bitstream we can start setting up the emulator. First we use the `emconfigutil` utility to define the system we want to emulate. In this case we want a single instance containing the platform we used.

In [5]:
!emconfigutil -f $platform --nd 1


****** configutil v2022.1 (64-bit)
  **** SW Build 3524075 on 2022-04-13-17:42:45
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.

INFO: [ConfigUtil 60-895]   Target platform: /opt/xilinx/platforms/xilinx_u250_gen3x16_xdma_2_1_202010_1/xilinx_u250_gen3x16_xdma_2_1_202010_1.xpfm
INFO: [ConfigUtil 60-1578]   This platform contains Xilinx Shell Archive '/opt/xilinx/platforms/xilinx_u250_gen3x16_xdma_2_1_202010_1/hw/hw.xsa'
INFO: [ConfigUtil 60-1032] Extracting hardware platform to ./.Xil/configutil-21474-alveo/hw
emulation configuration file `emconfig.json` is created in current working directory 


The `emconfig.json` file describes the platform and is used by the emulator runtime. default search path for this file is alongside the executable which would be `/usr/bin/python3` for a Python script. We can override this location by setting the `EMCONFIG_PATH` environment variable.

In [6]:
import os
os.environ['EMCONFIG_PATH'] = os.getcwd()

We also need to set the `XCL_EMULATION_MODE` environment variable to tell PYNQ and the underlying runtime to run against the emulator rather than real hardware. Note that this must be done before importing PYNQ.

In [7]:
os.environ['XCL_EMULATION_MODE'] = 'hw_emu'

## Running PYNQ on the Emulator

Now we can instantiate our overlay - this will start the emulator in the background

We can now use PYNQ in exactly the same way as if we were running against real hardware. For more details on the commands used refer to the _introduction_ notebook. The only change made here is to reduce the number of elements in the array to reduce the time it takes for the emulation to run.

In [8]:
import numpy as np
from pynq import Overlay, allocate

# Load Overlay
ol = Overlay("oop_emu.xclbin")

# Access the kernel
oop = ol.oop_1

node_dtype = np.dtype({
    'names': ['id', 'inV', 'outV', 'value'],
    'formats': [np.int32, np.int32, np.int32, np.int32]
})

N = 4  # number of nodes

# Create a Node object and an output buffer
nodes = allocate(shape=(N,), dtype=node_dtype)
out = allocate(shape=(1,), dtype=np.int32)

nodes[0]['id'] = 0
nodes[0]['inV'] = -1  # Set the 'inV' of the first node to -1 as None
nodes[0]['outV'] = 1
nodes[0]['value'] = 23
nodes[1]['id'] = 1
nodes[1]['inV'] = 0
nodes[1]['outV'] = 2
nodes[1]['value'] = 767
nodes[2]['id'] = 2
nodes[2]['inV'] = 1
nodes[2]['outV'] = 3  
nodes[2]['value'] = 544
nodes[3]['id'] = 3
nodes[3]['inV'] = 2
nodes[3]['outV'] = -2  # Set the 'outV' of the last node to -2 as None
nodes[3]['value'] = 989

nodes.sync_to_device()

# Call the kernel
oop.call(nodes, np.int32(2), out)  # Pass the starting ID as 0

nodes.sync_from_device()

# Print the result
print(out[0])


/home/jun/.local/lib/python3.8/site-packages/pydantic/_internal/_config.py:261: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


INFO: [HW-EMU 01] Hardware emulation runs simulation underneath. Using a large data set will result in long simulation times. It is recommended that a small dataset is used for faster execution. The flow uses approximate models for Global memories and interconnect and hence the performance data generated is approximate.
configuring penguin scheduler mode
scheduler config ert(0), dataflow(1), slots(16), cudma(1), cuisr(0), cdma(0), cus(1)
INFO: [HW-EMU 07-0] Please refer the path "/home/jun/pynq-notebooks/4-building-and-emulation/.run/18715/hw_em/device0/binary_0/behav_waveform/xsim/simulate.log" for more detailed simulation infos, errors and warnings.
989


In [9]:
erwrw

NameError: name 'erwrw' is not defined

In [ ]:
ol.free()

When using emulation it's important to close the device as well as free the bitstream to make the sure the simulator running the background is correctly shut down

In [ ]:
ol.free()
pynq.Device.active_device.close()

For more information on using the simulator refer the [Vitis debugging documentation](https://www.xilinx.com/html_docs/xilinx2019_2/vitis_doc/Chunk1273465831.html#pht1538574247013). In particular this will show how to

 * View waveforms of the accelerator execution
 * Debug the underlying hardware code interactively
 
Copyright (C) 2020 Xilinx, Inc